In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
    .enableHiveSupport()\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/26 17:27:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sqlTable = spark.read.format("jdbc") \
    .option("url", "jdbc:postgresql://metastore:5432/metastore") \
    .option("dbtable", "information_schema.tables") \
    .option("user", "jupyter") \
    .option("password", "jupyter") \
    .load()


In [4]:
sqlTable.show()

+-------------+------------+------------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|table_catalog|table_schema|        table_name|table_type|self_referencing_column_name|reference_generation|user_defined_type_catalog|user_defined_type_schema|user_defined_type_name|is_insertable_into|is_typed|commit_action|
+-------------+------------+------------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|    metastore|      public|               DBS|BASE TABLE|                        null|                null|                     null|                    null|                  null|               YES|      NO|         null|
|    metastore|      public|          DB_PRIVS|BASE TABLE|                        null|             

In [5]:
! hive -e 'CREATE TABLE inforamtion_table2 (\
  `table_catalog` string, \
  `table_schema` string, \
  `table_name` string, \
  `table_type` string, \
  `self_referencing_column_name` string, \
  `reference_generation` string,  \
  `user_defined_type_catalog` string,  \
  `user_defined_type_schema` string,  \
  `user_defined_type_name` string,  \
  `is_insertable_into rating` string,  \
  `is_typed` string,  \
  `commit_action` string  \
  ) \
  STORED AS ORC'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = e7c22575-c1a3-4cce-be85-d048c793522a

Logging initialized using configuration in jar:file:/opt/hive/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 34c59854-8255-4377-9a1b-4532337759c3
OK
Time taken: 0.9 seconds


In [6]:
sqlTable.createOrReplaceTempView("mytempTable") 

In [7]:
spark.sql("create table mytable as select * from mytempTable;");

24/12/26 17:35:53 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


In [8]:
sqlTable.write.mode("overwrite").format('orc').saveAsTable("default.inforamtion_table2")

In [10]:
q = spark.sql("select * from inforamtion_table2")
q.show()

+-------------+------------+------------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|table_catalog|table_schema|        table_name|table_type|self_referencing_column_name|reference_generation|user_defined_type_catalog|user_defined_type_schema|user_defined_type_name|is_insertable_into|is_typed|commit_action|
+-------------+------------+------------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|    metastore|      public|               DBS|BASE TABLE|                        null|                null|                     null|                    null|                  null|               YES|      NO|         null|
|    metastore|      public|          DB_PRIVS|BASE TABLE|                        null|             